In [2]:
import pandas as pd
from rdkit import Chem

In [17]:
data = pd.read_csv('sibur_element_119_final_train_data80.csv')

def is_valid_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False

data['is_valid'] = data['SMILES'].apply(is_valid_smiles)
valid_data = data[data['is_valid']].copy()
smiles_group = valid_data.groupby('SMILES')['LogP'].nunique().reset_index(name='num_unique_logP')
unique_smiles_with_varied_logP = smiles_group[smiles_group['num_unique_logP'] > 1]
unique_smiles_with_varied_logP[['SMILES']].to_csv('dublicate_SMILES.csv', index=False)
print("Датасет с уникальными SMILES, имеющими разные значения LogP, сохранён в 'unique_smiles_varied_logP.csv'")

distribution = unique_smiles_with_varied_logP['num_unique_logP'].value_counts().sort_index()
unique_smiles_without_duplicates = smiles_group[smiles_group['num_unique_logP'] == 1]
unique_data = valid_data[valid_data['SMILES'].isin(unique_smiles_without_duplicates['SMILES'])]
unique_data.to_csv('clean_smiles.csv', index=False)

[14:40:47] SMILES Parse Error: syntax error while parsing: CC(C)MC1=C(C(N=C2NNN=N2)=O)SC(OC)=C1Br
[14:40:47] SMILES Parse Error: check for mistakes around position 6:
[14:40:47] CC(C)MC1=C(C(N=C2NNN=N2)=O)SC(OC)=C1Br
[14:40:47] ~~~~~^
[14:40:47] SMILES Parse Error: Failed parsing SMILES 'CC(C)MC1=C(C(N=C2NNN=N2)=O)SC(OC)=C1Br' for input: 'CC(C)MC1=C(C(N=C2NNN=N2)=O)SC(OC)=C1Br'
[14:40:48] Explicit valence for atom # 8 N, 4, is greater than permitted
[14:40:48] Explicit valence for atom # 9 C, 5, is greater than permitted
[14:40:48] SMILES Parse Error: extra open parentheses while parsing: N(C(N=S)CC
[14:40:48] SMILES Parse Error: check for mistakes around position 2:
[14:40:48] N(C(N=S)CC
[14:40:48] ~^
[14:40:48] SMILES Parse Error: Failed parsing SMILES 'N(C(N=S)CC' for input: 'N(C(N=S)CC'
[14:40:48] SMILES Parse Error: syntax error while parsing: C1=CC2(C3C(CC4(C)CmO)(C(=O)CO)CCC4C3CCC2=CC1=O)=O)C
[14:40:48] SMILES Parse Error: check for mistakes around position 19:
[14:40:48] C1=CC2

Датасет с уникальными SMILES, имеющими разные значения LogP, сохранён в 'unique_smiles_varied_logP.csv'


[14:40:50] SMILES Parse Error: syntax error while parsing: OCC1C(C(C(O1)N2C=C(C)C(=N)NC2=O)O)Og
[14:40:50] SMILES Parse Error: check for mistakes around position 36:
[14:40:50] C=C(C)C(=N)NC2=O)O)Og
[14:40:50] ~~~~~~~~~~~~~~~~~~~~^
[14:40:50] SMILES Parse Error: Failed parsing SMILES 'OCC1C(C(C(O1)N2C=C(C)C(=N)NC2=O)O)Og' for input: 'OCC1C(C(C(O1)N2C=C(C)C(=N)NC2=O)O)Og'
[14:40:50] Explicit valence for atom # 13 Cl, 4, is greater than permitted
[14:40:50] SMILES Parse Error: extra open parentheses while parsing: O(C(NC=1C=CC(=CC=1)CC)=OC
[14:40:50] SMILES Parse Error: check for mistakes around position 2:
[14:40:50] O(C(NC=1C=CC(=CC=1)CC)=OC
[14:40:50] ~^
[14:40:50] SMILES Parse Error: Failed parsing SMILES 'O(C(NC=1C=CC(=CC=1)CC)=OC' for input: 'O(C(NC=1C=CC(=CC=1)CC)=OC'
[14:40:50] SMILES Parse Error: extra close parentheses while parsing: C1CCCCN1CCOC=O)NC=2C(=CC=CC=2)OCCCCC
[14:40:50] SMILES Parse Error: check for mistakes around position 15:
[14:40:50] C1CCCCN1CCOC=O)NC=2C(=CC=CC=

In [16]:
print("Распределение по количеству уникальных значений LogP для дублирующихся SMILES:")
print(distribution)

Распределение по количеству уникальных значений LogP для дублирующихся SMILES:
num_unique_logP
2    342
3    297
4    317
5     42
Name: count, dtype: int64


In [ ]:
!chemprop train --data-path train_filtred_unimap.csv --task-type regression --output-dir clean_smiles

In [ ]:
!chemprop predict --test-path dublicate_SMILES.csv --model-path clean_smiles/model_0/best.pt --preds-path pred_dublicate_SMILES.csv

In [21]:
data = pd.read_csv('sibur_element_119_final_train_data80.csv')
predictions = pd.read_csv('pred_dublicate_SMILES.csv')

def is_valid_smiles(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False

data['is_valid'] = data['SMILES'].apply(is_valid_smiles)
valid_data = data[data['is_valid']].copy()

smiles_group = valid_data.groupby('SMILES')['LogP'].nunique().reset_index(name='num_unique_logP')

duplicates = valid_data[valid_data['SMILES'].isin(smiles_group[smiles_group['num_unique_logP'] > 1]['SMILES'])]
duplicates = duplicates.merge(predictions, on='SMILES', suffixes=('_actual', '_predicted'))

duplicates['diff'] = (duplicates['LogP_actual'] - duplicates['LogP_predicted']).abs()
selected_duplicates = duplicates.loc[duplicates.groupby('SMILES')['diff'].idxmin()]

unique_smiles = smiles_group[smiles_group['num_unique_logP'] == 1]['SMILES']
unique_data = valid_data[valid_data['SMILES'].isin(unique_smiles)]

final_data = pd.concat([unique_data, selected_duplicates[['SMILES', 'LogP_actual']].rename(columns={'LogP_actual': 'LogP'})])

final_data[['SMILES', 'LogP']].to_csv('final_fixed_train.csv', index=False)

[14:43:44] SMILES Parse Error: syntax error while parsing: CC(C)MC1=C(C(N=C2NNN=N2)=O)SC(OC)=C1Br
[14:43:44] SMILES Parse Error: check for mistakes around position 6:
[14:43:44] CC(C)MC1=C(C(N=C2NNN=N2)=O)SC(OC)=C1Br
[14:43:44] ~~~~~^
[14:43:44] SMILES Parse Error: Failed parsing SMILES 'CC(C)MC1=C(C(N=C2NNN=N2)=O)SC(OC)=C1Br' for input: 'CC(C)MC1=C(C(N=C2NNN=N2)=O)SC(OC)=C1Br'
[14:43:44] Explicit valence for atom # 8 N, 4, is greater than permitted
[14:43:44] Explicit valence for atom # 9 C, 5, is greater than permitted
[14:43:44] SMILES Parse Error: extra open parentheses while parsing: N(C(N=S)CC
[14:43:44] SMILES Parse Error: check for mistakes around position 2:
[14:43:44] N(C(N=S)CC
[14:43:44] ~^
[14:43:44] SMILES Parse Error: Failed parsing SMILES 'N(C(N=S)CC' for input: 'N(C(N=S)CC'
[14:43:44] SMILES Parse Error: syntax error while parsing: C1=CC2(C3C(CC4(C)CmO)(C(=O)CO)CCC4C3CCC2=CC1=O)=O)C
[14:43:44] SMILES Parse Error: check for mistakes around position 19:
[14:43:44] C1=CC2